<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

## Tackling an NLP Problem with Naive Bayes

----

We can sketch out the data science process as follows:
1. Define the problem.
2. Obtain the data.
3. Explore the data.
4. Model the data.
5. Evaluate the model.
6. Answer the problem.

**Instructor's note: While I completely agree with the idea expressed below, we will be discussing Naive Bayes in class**

In this lab, we are going to apply a **new** modeling technique to natural language processing data.

> "But how can we apply a modeling technique we haven't learned?!"

The DSI program is great - but we can't teach you *everything* about data science in 12 weeks! This lab is designed to help you start learning something new without it being taught in a formal lesson. 

- Later in the cohort (like for your capstone!), you'll be exploring models, libraries, and resources that you haven't been explicitly taught.
- After the program, you'll want to continue developing your skills. Being comfortable with documentation and being confident in your ability to read something new and decide whether or not it is an appropriate method for the problem you're trying to solve is **incredibly** valuable.

### Step 1: Define the problem.

Many organizations have a substantial interest in classifying users of their product into groups. Some examples:
- A company that serves as a marketplace may want to predict who is likely to purchase a certain type of product on their platform, like books, cars, or food.
- An application developer may want to identify which individuals are willing to pay money for "bonus features" or to upgrade their app.
- A social media organization may want to identify who generates the highest rate of content that later goes "viral."

### Summary
In this lab, you're an engineer for Facebook. In recent years, the organization Cambridge Analytica gained worldwide notoriety for its use of Facebook data in an attempt to sway electoral outcomes.

Cambridge Analytica, an organization staffed with lots of Ph.D. researchers, used the Big5 personality groupings (also called OCEAN) to group people into one of 32 different groups.
- The five qualities measured by this personality assessment are:
    - **O**penness
    - **C**onscientiousness
    - **E**xtroversion
    - **A**greeableness
    - **N**euroticism
- Each person could be classified as "Yes" or "No" for each of the five qualities.
- This makes for 32 different potential combinations of qualities. ($2^5 = 32$)
- You don't have to check it out, but if you want to learn more about this personality assessment, head to [**the Wikipedia page**](https://en.wikipedia.org/wiki/Big_Five_personality_traits).
- There's also [**a short (3-4 pages) academic paper describing part of this approach**](./celli-al_wcpr13.pdf).

Cambridge Analytica's methodology was, roughly, the following:
- Gather a large amount of data from Facebook.
- Use this data to predict an individual's Big5 personality "grouping."
- Design political advertisements that would be particularly effective to that particular "grouping." (For example, are certain advertisements particularly effective toward people with specific personality traits?)

You want to know the **real-world problem**: "Is what Cambridge Analytica attempted to do actually possible, or is it junk science?"

However, we'll solve the related **data science problem**: "Are one's Facebook statuses predictive of whether or not one is agreeable?"
> Note: If Facebook statuses aren't predictive of one being agreeable (one of the OCEAN qualities), then Cambridge Analytica's approach won't work very well!

### Step 2: Obtain the data.

Obviously, there are plenty of opportunities to discuss the ethics surrounding this particular issue... so let's do that.

In [5]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.pipeline import Pipeline

In [6]:
data = pd.read_csv('./mypersonality_final.csv', encoding = 'ISO-8859-1')

In [ ]:
data.head()

**1. What is the difference between anonymity and confidentiality? All else held equal, which tends to keep people safer?**

**Answer:** Anonymity is that when information collected from research, they would not collect identifying information of individual persons. Confidentiality maintains confidentiality of information collected from research participants. It can only guarantee the safe by not collecting any personally identifying information which is Anonymity.

**2. Suppose that the "unique identifier" in the above data, the `#AUTHID`, is a randomly generated key so that it can never be connected back to the original poster. Have we guaranteed anonymity here? Why or why not?**

**Answer:** To garantee anonymity, anyname in the Status must be replace or remove sunce there are possibilities that users would put his or her name or people they are related to the STaTUS.

**3. As an engineer for Facebook, you recognize that user data will be used by Facebook and by other organizations - that won't change. However, what are at least three recommendations you would bring to your manager to improve how data is used and shared? Be as specific as you can.**

**Answer:**
1. Ensure that shared data is anonymized and aggregated wherever possible to minimize the risk of personal identification.
2. Asking consent for data sharing, ensuring users are actively involved in decisions about their data.Provide clear resources about why consent is needed, how data is used, and the benefits/risks involved.
3. Redesign the consent process to be tiered and specific to different types of data and use cases, with clear language and contextual prompts.

### Step 3: Explore the data.

- Note: For our $X$ variable, we will only use the `STATUS` variable. For our $Y$ variable, we will only use the `cAGR` variable.

**4. Explore the data here.**
> We aren't explicitly asking you to do specific EDA here, but what EDA would you generally do with this data? Do the EDA you usually would, especially if you know what the goal of this analysis is.

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9917 entries, 0 to 9916
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   #AUTHID       9917 non-null   object 
 1   STATUS        9917 non-null   object 
 2   sEXT          9917 non-null   float64
 3   sNEU          9917 non-null   float64
 4   sAGR          9917 non-null   float64
 5   sCON          9917 non-null   float64
 6   sOPN          9917 non-null   float64
 7   cEXT          9917 non-null   object 
 8   cNEU          9917 non-null   object 
 9   cAGR          9917 non-null   object 
 10  cCON          9917 non-null   object 
 11  cOPN          9917 non-null   object 
 12  DATE          9917 non-null   object 
 13  NETWORKSIZE   9917 non-null   float64
 14  BETWEENNESS   9917 non-null   float64
 15  NBETWEENNESS  9917 non-null   float64
 16  DENSITY       9917 non-null   float64
 17  BROKERAGE     9917 non-null   float64
 18  NBROKERAGE    9917 non-null 

In [17]:
data.isnull().sum()

#AUTHID         0
STATUS          0
sEXT            0
sNEU            0
sAGR            0
sCON            0
sOPN            0
cEXT            0
cNEU            0
cAGR            0
cCON            0
cOPN            0
DATE            0
NETWORKSIZE     0
BETWEENNESS     0
NBETWEENNESS    0
DENSITY         0
BROKERAGE       0
NBROKERAGE      0
TRANSITIVITY    1
dtype: int64

In [18]:
data.shape

(9917, 20)

In [19]:
X = data['STATUS']
y = data['cAGR']

In [20]:
X.shape , y.shape

((9917,), (9917,))

In [21]:
y.value_counts(normalize=True)

cAGR
y    0.531209
n    0.468791
Name: proportion, dtype: float64

**5. What is the difference between CountVectorizer and TFIDFVectorizer?**

**Answer:** CountVectorizer counts word occurances which is easier to understand. While TFIDFVectorizer balances word 
frequency with importance which can reduces impact of common but unimportant words.

In term of use cases, CountVectorizer is In term of use cases, CountVectorizer suits for small to medium datasets, when you need to understand exact word frequencies and topic modeling tasks. On the other hand, TFIDFVectorizer fits more on classification tasks where term uniqueness matters or search systems.

**6. What are stopwords?**

**Answer:** Stopwords are common words that typically don't contribute meaningful information to text analysis.

**7. Give an example of when you might remove stopwords.**

**Answer:** Examples of stopwords are “the,” “and,” “is,” “in,” “for,” and “it.” These stopwords are frequently removed to focus on more meaningful terms when processing text data in natural language processing tasks such as text classification or sentiment analysis.

**8. Give an example of when you might keep stopwords in your model.**

**Answer:** Some stopwords might be meaningful in specific industries or domain. Removing these stopwords might effect the accurate of model. For example, in social media the word 'like' have its own meaning.

### Step 4: Model the data.

We are going to fit two types of models: a logistic regression and a [**Naive Bayes classifier**](https://scikit-learn.org/stable/modules/naive_bayes.html).

**Reminder:** We will only use the feature `STATUS` to model `cAGR`.

### We want to attempt to fit our models on sixteen sets of features:

1. CountVectorizer with 100 features, with English stopwords removed and with an `ngram_range` that includes 1 and 2.
2. CountVectorizer with 100 features, with English stopwords removed and with the default `ngram_range`.
3. CountVectorizer with 100 features, with English stopwords kept in and with an `ngram_range` that includes 1 and 2.
4. CountVectorizer with 100 features, with English stopwords kept in and with the default `ngram_range`.
5. CountVectorizer with 500 features, with English stopwords removed and with an `ngram_range` that includes 1 and 2.
6. CountVectorizer with 500 features, with English stopwords removed and with the default `ngram_range`.
7. CountVectorizer with 500 features, with English stopwords kept in and with an `ngram_range` that includes 1 and 2.
8. CountVectorizer with 500 features, with English stopwords kept in and with the default `ngram_range`.
9. TFIDFVectorizer with 100 features, with English stopwords removed and with an `ngram_range` that includes 1 and 2.
10. TFIDFVectorizer with 100 features, with English stopwords removed and with the default `ngram_range`.
11. TFIDFVectorizer with 100 features, with English stopwords kept in and with an `ngram_range` that includes 1 and 2.
12. TFIDFVectorizer with 100 features, with English stopwords kept in and with the default `ngram_range`.
13. TFIDFVectorizer with 500 features, with English stopwords removed and with an `ngram_range` that includes 1 and 2.
14. TFIDFVectorizer with 500 features, with English stopwords removed and with the default `ngram_range`.
15. TFIDFVectorizer with 500 features, with English stopwords kept in and with an `ngram_range` that includes 1 and 2.
16. TFIDFVectorizer with 500 features, with English stopwords kept in and with the default `ngram_range`.

**9. Rather than manually instantiating 16 different vectorizers, what `sklearn` class have we learned about that might make this easier? Use it.**

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,stratify=y, random_state=42)

In [33]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((7437,), (2480,), (7437,), (2480,))

In [73]:
# Create Pipeline
pipe = Pipeline([
    ('cvec', CountVectorizer()),  
    ('lr', LogisticRegression())
])

# Hyperparameters
params = {
    'cvec__max_features': [100, 500,1_000,700,1_500,2_000,2_500],  
    'cvec__stop_words': ['english', None],
    'cvec__ngram_range': [(1, 2), (1, 1)] 
}

In [75]:
# Instantiate GridSearchCV
gs = GridSearchCV(pipe, param_grid=params, cv=5)
# Fit
gs.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('cvec', CountVectorizer()),
                                       ('lr', LogisticRegression())]),
             param_grid={'cvec__max_features': [100, 500, 1000, 700, 1500, 2000,
                                                2500],
                         'cvec__ngram_range': [(1, 2), (1, 1)],
                         'cvec__stop_words': ['english', None]})

In [76]:
print(f" Best score: {gs.best_score_}")
print(f" Best parameters: {gs.best_params_}")

 Best score: 0.5771138396569553
 Best parameters: {'cvec__max_features': 2500, 'cvec__ngram_range': (1, 2), 'cvec__stop_words': 'english'}


**10. What are some of the advantages of fitting a logistic regression model?**

**Answer:** Logistic regression model gives us the probabilities of being in each class and interpretign the result

**11. Fit a logistic regression model and compare it to the baseline.**

In [98]:
# Baseline
y_train.value_counts(normalize = True).round(5)

cAGR
y    0.53126
n    0.46874
Name: proportion, dtype: float64

In [94]:
# Fit your model
pipe.fit(X_train, y_train)
# Training score
print(f'Training score: {round(pipe.score(X_train, y_train),5)}')
# Test score
print(f'Testing score: {round(pipe.score(X_test, y_test),5)}')

Training score: 0.91489
Testing score: 0.58387


### Summary of Naive Bayes 

Naive Bayes is a classification technique that relies on probability to classify observations.
- It's based on a probability rule called **Bayes' Theorem**... thus, "**Bayes**."
- It makes an assumption that isn't often met, so it's "**naive**."

Despite being a model that relies on a naive assumption, it often performs pretty well! (This is kind of like linear regression... we aren't always guaranteed homoscedastic errors in linear regression, but the model might still do a good job regardless.)
- [**Interested in the details?**](https://www.cs.unb.ca/~hzhang/publications/FLAIRS04ZhangH.pdf)


The [**sklearn documentation**](https://scikit-learn.org/stable/modules/naive_bayes.html) is here, but it can be intimidating. So, to quickly summarize the Bayes and Naive parts of the model...

#### Bayes' Theorem
If you've seen Bayes' Theorem, it relates the probability of $P(A|B)$ to $P(B|A)$. (Don't worry; we won't be doing any probability calculations by hand! However, you may want to refresh your memory on conditional probability from our earlier lessons if you forget what a conditional probability is.)

$$
\begin{eqnarray*}
\text{Bayes' Theorem: } P(A|B) &=& \frac{P(B|A)P(A)}{P(B)}
\end{eqnarray*}
$$

- Let $A$ be that someone is "agreeable," like the OCEAN category.
- Let $B$ represent the words used in their Facebook post.

$$
\begin{eqnarray*}
\text{Bayes' Theorem: } P(A|B) &=& \frac{P(B|A)P(A)}{P(B)} \\
\Rightarrow P(\text{person is agreeable}|\text{words in Facebook post}) &=& \frac{P(\text{words in Facebook post}|\text{person is agreeable})P(\text{person is agreeable})}{P(\text{words in Facebook post})}
\end{eqnarray*}
$$

We want to calculate the probability that someone is agreeable **given** the words that they used in their Facebook post! (Rather than calculating this probability by hand, this is done under the hood and we can just see the results by checking `.predict_proba()`.) However, this is exactly what our model is doing. We can (a.k.a. the model can) calculate the pieces on the right-hand side of the equation to give us a probability estimate of how likely someone is to be agreeable given their Facebook post.

#### Naive Assumption

If our goal is to estimate $P(\text{person is agreeable}|\text{words in Facebook post})$, that can be quite tricky.

---

<details><summary>Bonus: if you want to understand why that's complicated, click here.</summary>
    
- The event $\text{"words in Facebook post"}$ is a complicated event to calculate.

- If a Facebook post has 100 words in it, then the event $\text{"words in Facebook post"} = \text{"word 1 is in the Facebook post" and "word 2 is in the Facebook post" and }\ldots \text{ and "word 100 is in the Facebook post"}$.

- To calculate the joint probability of all 100 words being in the Facebook post gets complicated pretty quickly. (Refer back to the probability notes on how to calculate the joint probability of two events if you want to see more.)
</details>

---

To simplify matters, we make an assumption: **we assume that all of our features are independent of one another.**

In some contexts, this assumption might be realistic!

**12. Why would this assumption not be realistic with NLP data?**

**Answer:** Since realitiy words might also be related to the thing they’re describing,the words in the sentence are not always necessarily independent of one another. Moreover,this could lead to under or overestimate the conditional probabilities of individual words. 

Despite this assumption not being realistic with NLP data, we still use Naive Bayes pretty frequently.
- It's a very fast modeling algorithm. (which is great especially when we have lots of features and/or lots of data!)
- It is often an excellent classifier, outperforming more complicated models.

There are three common types of Naive Bayes models: Bernoulli Naive Bayes, Multinomial Naive Bayes, and Gaussian Naive Bayes.
- How do we pick which of the three models to use? It depends on our $X$ variable.
    - Bernoulli Naive Bayes is appropriate when our features are all 0/1 variables.
        - [**Bernoulli NB Documentation**](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.BernoulliNB.html#sklearn.naive_bayes.BernoulliNB)
    - Multinomial Naive Bayes is appropriate when our features are variables that take on only positive integer counts.
        - [**Multinomial NB Documentation**](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html#sklearn.naive_bayes.MultinomialNB)
    - Gaussian Naive Bayes is appropriate when our features are Normally distributed variables. (Realistically, though, we kind of use Gaussian whenever neither Bernoulli nor Multinomial works.)
        - [**Gaussian NB Documentation**](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html#sklearn.naive_bayes.GaussianNB)

**13. Suppose you CountVectorized your features. Which Naive Bayes model would be most appropriate to fit? Why? Fit it.**

**Answer:** The most appropriate model to fit is Multinomial NB Documentation. The reason is that our feature is word counts which is expected to be only positive integer counts.

In [113]:
# Create Pipeline
pipe_cvec = Pipeline([
    ('cvec', CountVectorizer()),  
    ('nb', MultinomialNB())
])

# Hyperparameters
params_cvec = {
    'cvec__max_features': [100, 500,1_000,700,1_500,2_000,2_500],  
    'cvec__stop_words': ['english', None],
    'cvec__ngram_range': [(1, 2), (1, 1)] 
}

# Instantiate GridSearchCV
gs_cvec = GridSearchCV(pipe_cvec, param_grid=params_cvec, cv=5)
# Fit
gs_cvec.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('cvec', CountVectorizer()),
                                       ('nb', MultinomialNB())]),
             param_grid={'cvec__max_features': [100, 500, 1000, 700, 1500, 2000,
                                                2500],
                         'cvec__ngram_range': [(1, 2), (1, 1)],
                         'cvec__stop_words': ['english', None]})

In [119]:
print(f" CVEC Best parameters: {gs_cvec.best_params_}")

 CVEC Best parameters: {'cvec__max_features': 2500, 'cvec__ngram_range': (1, 2), 'cvec__stop_words': None}


**14. Suppose you TFIDFVectorized your features. Which Naive Bayes model would be most appropriate to fit? Why? Fit it.**

**Answer:** 

In [134]:
# Create Pipeline
pipe_tfid = Pipeline([
    ('tfid', TfidfVectorizer()),  
    ('nb', MultinomialNB())
])

# Hyperparameters
params_tfid = {  
    'tfid__stop_words': ['english', None],
    'tfid__ngram_range': [(1, 2), (1, 1)] 
}

# Instantiate GridSearchCV
gs_tfid = GridSearchCV(pipe_tfid, param_grid=params_tfid, cv=5)
# Fit
gs_tfid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfid', TfidfVectorizer()),
                                       ('nb', MultinomialNB())]),
             param_grid={'tfid__ngram_range': [(1, 2), (1, 1)],
                         'tfid__stop_words': ['english', None]})

In [135]:
print(f" TFID Best parameters: {gs_tfid.best_params_}")

 TFID Best parameters: {'tfid__ngram_range': (1, 2), 'tfid__stop_words': 'english'}


**15. Compare the performance of your models.**

In [137]:
print(f" CVEC Best score: {gs_cvec.best_score_}")
print(f" TFID Best score: {gs_tfid.best_score_}")

 CVEC Best score: 0.5759055180742059
 TFID Best score: 0.5991684202153429


In [ ]:
# TFID has better score.

**16. Even though we didn't explore the full extent of Cambridge Analytica's modeling, based on what we did here, how effective was their approach at using Facebook data to model agreeableness?**

In [ ]:
# I thinks that It is very hard to predict someone personality from the status. 
# Since those status might be their thought in a moment.